In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from AGS.memory import AGSMemory
from AGS.reasoning import AGSReasoner
from AGS.retrieval import AGSRetriever
from AGS.voice import VoiceProcessor
from AGS.config import AGSConfig
import json, os

# Load config and initialize components
AGSConfig().load_env()
reasoner = AGSReasoner()
voice = VoiceProcessor()

DATASET_DIR = "datasets"
OUTPUT_DIR = "output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

dataset_files = sorted([f for f in os.listdir(DATASET_DIR) if f.endswith(".json")])

for idx, filename in enumerate(dataset_files, 1):
    with open(os.path.join(DATASET_DIR, filename)) as f:
        data = json.load(f)

    # Extract user query
    user_query = data.get("question") or data.get("prompt") or str(data)

    # 🤖 Reasoning output
    explanation = reasoner.explain_code(user_query)

    # 🔊 Voice output
    audio_url = voice.speak(explanation)

    # Save output
    output_data = {
        "input": data,
        "reasoning": explanation,
        "voice": audio_url
    }

    with open(f"{OUTPUT_DIR}/{filename.replace('.json', '_response.json')}", "w") as out_f:
        json.dump(output_data, out_f, indent=2)

    print(f"✅ Processed {idx:03}: {filename}")

ModuleNotFoundError: No module named 'AGS'

In [ ]:
import os
from dotenv import load_dotenv

class AGSConfig:
    def load_env(self):
        load_dotenv()
        self.OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
        self.ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
        self.ELEVENLABS_VOICE_ID = os.getenv("ELEVENLABS_VOICE_ID")

In [ ]:
import openai
import os

class AGSReasoner:
    def __init__(self):
        openai.api_key = os.getenv("OPENAI_API_KEY")

    def explain_code(self, prompt):
        messages = [
            {"role": "system", "content": "You're a reasoning AI that explains logic clearly."},
            {"role": "user", "content": prompt}
        ]
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=messages,
            temperature=0.5
        )
        return response.choices[0].message["content"]

In [ ]:
import requests
import os

class VoiceProcessor:
    def __init__(self):
        self.api_key = os.getenv("ELEVENLABS_API_KEY")
        self.voice_id = os.getenv("ELEVENLABS_VOICE_ID")

    def speak(self, text):
        url = f"https://api.elevenlabs.io/v1/text-to-speech/{self.voice_id}"
        headers = {
            "xi-api-key": self.api_key,
            "Content-Type": "application/json"
        }
        data = {
            "text": text,
            "model_id": "eleven_monolingual_v1",
            "voice_settings": {"stability": 0.5, "similarity_boost": 0.75}
        }

        response = requests.post(url, json=data, headers=headers)
        if response.status_code == 200:
            out_path = f"output/audio_{hash(text)}.mp3"
            with open(out_path, "wb") as f:
                f.write(response.content)
            return out_path
        else:
            return "Voice generation failed."

In [ ]:
pip install openai python-dotenv requests

In [ ]:
from AGS.memory import AGSMemory
from AGS.reasoning import AGSReasoner
from AGS.voice import VoiceProcessor
from AGS.config import AGSConfig